In [1]:
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
import pickle
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.5/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
mypath_input = '../../datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/all_events.json'

In [3]:
mypath_output = "/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/padded_arrays/"

In [4]:
df = pd.read_json(mypath_input).reset_index(drop = True)

In [7]:
df = df.rename(columns = {'events':'event'})

In [9]:
df.head(2)

,HADM_ID,HOSPITAL_EXPIRE_FLAG,SUBJECT_ID,event,inputevents_cv,inputevents_mv,labevents,microbioevents,noteevents,outputevents,prescriptionevents,procedureevents
0,163353,0,2,"[Resp, Rate, 3603-39-Breath, Resp, Rate, 3603-...",[NaN],[NaN],"[Atypical, Lymphocytes, Blood, Hematology, 511...","[BLOOD, CULTURE, -, NEONATE, Negative, Culture]","[Nursing/other, Report, Nursing, Transfer, not...",[NaN],"[MAIN, NEOIVGentamicin, NaN, DRUG_CD-GENT10I, ...",[NaN]
1,145834,0,3,"[Sputum, Source, 657-Suctioned, Riker, SAS, Sc...","[D5W, Intravenous, Push, 30013-250-ml, D5W, In...",[NaN],"[Anion, Gap, Blood, Chemistry, 50868-17-mEq/L,...","[SWAB, PROBABLE, ENTEROCOCCUS, SWAB, YEAST, SW...","[Echo, Report, PATIENT/TEST, INFORMATION:, Ind...","[Urine, Out, Foley, 40055-20-ml, Urine, Out, F...",[NaN],[NaN]


In [10]:
df[df['HOSPITAL_EXPIRE_FLAG']==1].head(2)

,HADM_ID,HOSPITAL_EXPIRE_FLAG,SUBJECT_ID,event,inputevents_cv,inputevents_mv,labevents,microbioevents,noteevents,outputevents,prescriptionevents,procedureevents
20,134748,1,1744,"[RU, Strength/Movement, 596-Moves on Bed, Reas...","[D5W, Intravenous, Push, 30013-50-ml, D5W, Int...",[NaN],"[Bacteria, Urine, Hematology, Bilirubin, Urine...","[BLOOD, CULTURE, Negative, Culture, BLOOD, CUL...","[ECG, Report, Sinus, rhythm, Nondiagnostic, la...","[Pre-Admission, Output, Pre-Admission, Output,...",[NaN],[NaN]
21,139336,1,17224,"[Ectopy, Type, 161-None, Skin, Integrity, 644-...","[.9%, Normal, Saline, Intravenous, Push, 30018...",[NaN],"[Acetaminophen, Blood, Chemistry, Anion, Gap, ...","[BLOOD, CULTURE, Negative, Culture, URINE, Neg...","[Discharge, summary, Report, Admission, Date:,...","[Pre-Admission, Output, Pre-Admission, Output,...","[MAIN, Heparin, Flush, CVL, 100, units/ml, Hep...",[NaN]


In [11]:
# read ICUTAYS.csv
df_icu =  pd.read_csv('../../datasets/raw/ICUSTAYS.csv')

In [12]:
df_icu.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [13]:
#dropping ICUstays with LOS<1
df_icu = df_icu[df_icu['LOS'] > 1]

In [14]:
len(df.HADM_ID.unique())

53482

In [15]:
df = df[df['HADM_ID'].isin(df_icu['HADM_ID'].unique())]
df = df.reset_index(drop=True)

In [16]:
df = df.merge(df_icu[['HADM_ID','LOS']], how='left',left_on='HADM_ID', right_on='HADM_ID')

In [17]:
df.head()

,HADM_ID,HOSPITAL_EXPIRE_FLAG,SUBJECT_ID,event,inputevents_cv,inputevents_mv,labevents,microbioevents,noteevents,outputevents,prescriptionevents,procedureevents,LOS
0,145834,0,3,"[Sputum, Source, 657-Suctioned, Riker, SAS, Sc...","[D5W, Intravenous, Push, 30013-250-ml, D5W, In...",[NaN],"[Anion, Gap, Blood, Chemistry, 50868-17-mEq/L,...","[SWAB, PROBABLE, ENTEROCOCCUS, SWAB, YEAST, SW...","[Echo, Report, PATIENT/TEST, INFORMATION:, Ind...","[Urine, Out, Foley, 40055-20-ml, Urine, Out, F...",[NaN],[NaN],6.0646
1,185483,0,9078,"[Riker, SAS, Scale, 1337-Sedated, Riker, SAS, ...","[D5W, Intravenous, Push, 30013-250-ml, D5W, In...",[NaN],"[Eosinophils, Blood, Hematology, 51200-0-%, He...","[SPUTUM, YEAST, BLOOD, CULTURE, Negative, Cult...","[ECG, Report, Atrial, fibrillation, Inferior, ...","[Gastric, Nasogastric, 40052-150-ml, Pre-Admis...","[MAIN, Potassium, Phosphate, NaN, DRUG_CD-KPHO...",[NaN],9.7236
2,167537,0,9162,"[RU, Strength/Movement, 596-Normal Strength, R...","[D5W, Intravenous, Push, 30013-250-ml, .9%, No...",[NaN],"[Acetaminophen, Blood, Chemistry, Amylase, Blo...",[NaN],"[Radiology, CT, HEAD, W/O, CONTRAST, 7:24, PM,...","[Urine, Out, Foley, 40055-120-ml, Urine, Out, ...","[MAIN, Insulin, Insulin, -, Sliding, Scale, DR...",[NaN],1.8494
3,100006,0,9895,"[Assistance, Device, 54-1 Person Assist, RUL, ...","[Po, Intake, Oral, 30056-120-ml, Po, Intake, O...",[NaN],"[Calculated, Total, CO2, Blood, Blood, Gas, 50...",[NaN],"[ECG, Report, Sinus, tachycardia, Left, axis, ...","[Urine, Out, Void, 40069-400-ml, Urine, Out, V...","[MAIN, Azithromycin, Azithromycin, DRUG_CD-ZIT...",[NaN],4.9776
4,135351,0,17213,"[Color, Overall, 3373-Pink, Bowel, Sounds, 80-...","[E, 20, FS, PO, By, Mouth, 30270-30-ml, E, 20,...",[NaN],"[MCV, Blood, Hematology, 51250-107-fL, abnorma...","[BLOOD, CULTURE, -, NEONATE, Negative, Culture]","[Nursing/other, Report, Attending, Note, Day, ...","[Urine, ., 43175-3-ml, Urine, ., 43175-2-ml, U...",[NaN],[NaN],1.9599


In [18]:
# remove records with LOS missing
df = df[df['LOS'].notnull()]

In [19]:
# create binary LOS i.e. >7 or 7
df['LOS'] = df['LOS'].map(lambda x: 0 if x < 7 else 1)

In [20]:
print("number of patients {}".format(len(df.SUBJECT_ID.unique())))
print("number of admissions {}".format(len(df.HADM_ID.unique())))
print("number of in-hospital mortality {}".format(sum(df['HOSPITAL_EXPIRE_FLAG'])))
print('patient with atleast 7 days icu stay {}'.format(sum(df['LOS'])))

number of patients 35348
number of admissions 42818
number of in-hospital mortality 4146
patient with atleast 7 days icu stay 8450


In [21]:
#split SubjectID into train/valid/test
X_train, X_test = train_test_split(df.SUBJECT_ID.unique(),test_size=0.15, random_state=1234)

X_train, X_valid = train_test_split(X_train,test_size=0.15, random_state=1234)

print(len(X_train),len(X_valid),len(X_test))

25538 4507 5303


In [22]:
# Export the list of hospital admission id for train , valid, test
# Save SUBJECT_ID for training data
import json
with open(mypath_output+'trainlist_SUBJECT_ID.json', 'w') as filehandle:
    json.dump(X_train.tolist(), filehandle)
    
# Save HADM_ID for training data
import json
with open(mypath_output+'validlist_SUBJECT_ID.json', 'w') as filehandle:
    json.dump(X_valid.tolist(), filehandle)

# Save HADM_ID for training data
import json
with open(mypath_output+'testlist_SUBJECT_ID.json', 'w') as filehandle:
    json.dump(X_test.tolist(), filehandle)

In [23]:
print("The ratio of in-hospital mortality in train, valid and test are {:.2} {:.2} {:.2}".format(
    sum(df[df.SUBJECT_ID.isin(X_train)]['HOSPITAL_EXPIRE_FLAG'])/len(X_train),
    sum(df[df.SUBJECT_ID.isin(X_valid)]['HOSPITAL_EXPIRE_FLAG'])/len(X_valid),
    sum(df[df.SUBJECT_ID.isin(X_test)]['HOSPITAL_EXPIRE_FLAG'])/len(X_test)))

The ratio of in-hospital mortality in train, valid and test are 0.12 0.12 0.12


In [24]:
print("The ratio of icustays atleast 7 days in train, valid and test are {:.2} {:.2} {:.2}".format(
    sum(df[df.SUBJECT_ID.isin(X_train)]['LOS'])/len(X_train),
    sum(df[df.SUBJECT_ID.isin(X_valid)]['LOS'])/len(X_valid),
    sum(df[df.SUBJECT_ID.isin(X_test)]['LOS'])/len(X_test)))

The ratio of icustays atleast 7 days in train, valid and test are 0.24 0.24 0.24


In [25]:
# takes input as a pd.series of lists and determines longest length of a list in the series
def cal_max_len(x):
    a = []
    for i in range(len(x)):
        a.append(len(x[i]))
    max_len = int(max(a))
    pctl_999 = int(np.percentile(a, 99.9))
    return max_len, pctl_999

In [26]:
#create dictionary to hold max length of list in each column
max_len_dict = {}
pctl_999_dict = {}
for i in df.columns:
    if 'event' in i:
        max_len_dict.update({i:cal_max_len(df[i])[0]})
        pctl_999_dict.update({i:cal_max_len(df[i])[1]})

In [27]:
max_len_dict

{'event': 24791,
 'inputevents_cv': 8325,
 'inputevents_mv': 11623,
 'labevents': 24991,
 'microbioevents': 1502,
 'noteevents': 32149,
 'outputevents': 1153,
 'prescriptionevents': 7271,
 'procedureevents': 246}

In [28]:
pctl_999_dict

{'event': 11206,
 'inputevents_cv': 2472,
 'inputevents_mv': 2747,
 'labevents': 6534,
 'microbioevents': 348,
 'noteevents': 19420,
 'outputevents': 433,
 'prescriptionevents': 2525,
 'procedureevents': 144}

In [29]:
print(max_len_dict)
# Save dict containing max length of each event type for training data
import json
with open(mypath_output+'max_padlen_dict.json', 'w') as filehandle:
    json.dump(max_len_dict, filehandle)

{'microbioevents': 1502, 'inputevents_mv': 11623, 'prescriptionevents': 7271, 'outputevents': 1153, 'inputevents_cv': 8325, 'procedureevents': 246, 'noteevents': 32149, 'event': 24791, 'labevents': 24991}


In [30]:
pctl_999_dict
print(pctl_999_dict)
# Save dict containing 99.9 pctl length of each event type for training data
import json
with open(mypath_output+'pctl999_padlen_dict.json', 'w') as filehandle:
    json.dump(pctl_999_dict, filehandle)

{'microbioevents': 348, 'inputevents_mv': 2747, 'prescriptionevents': 2525, 'outputevents': 433, 'inputevents_cv': 2472, 'procedureevents': 144, 'noteevents': 19420, 'event': 11206, 'labevents': 6534}


### Create and Fit Tokenizer for Train Data

In [53]:
# Creating training, valid, test Dataset
df_train = df[df.SUBJECT_ID.isin(X_train)].reset_index(drop = True)
df_valid = df[df.SUBJECT_ID.isin(X_valid)].reset_index(drop = True)
df_test = df[df.SUBJECT_ID.isin(X_test)].reset_index(drop = True)

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [33]:
# create output path
tokenizer_path = "/home/jupyter/output/tokenizer/"
import os
os.makedirs(tokenizer_path, exist_ok=True)

In [38]:
%%time
#Training tokenizers on train dataset
tokenizer_dict = {} 
for i in df.columns:
    if 'event' in i:
        # initiate tokenizer
        t = Tokenizer(lower=True,split=',', filters= '')
        # Fit tokenizer on event
        t.fit_on_texts(df_train[i])
        tokenizer_dict.update({i:t})
        print("Tokenizer built for {}".format(i))
print('All tokenizers are built')

Tokenizer built for event
Tokenizer built for inputevents_cv
Tokenizer built for inputevents_mv
Tokenizer built for labevents
Tokenizer built for microbioevents
Tokenizer built for noteevents
Tokenizer built for outputevents
Tokenizer built for prescriptionevents
Tokenizer built for procedureevents
All tokenizers are built
CPU times: user 4min 33s, sys: 528 ms, total: 4min 33s
Wall time: 4min 33s


In [39]:
#saving the Chartevents tokenizer to disk
with open(tokenizer_path+"tokenizer_chartevents.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['event'],handle)  
    
#saving the Inputevents_cv tokenizer to disk
with open(tokenizer_path+"tokenizer_inputevents_cv.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['inputevents_cv'],handle) 
    
#saving the Inputevents_mv tokenizer to disk
with open(tokenizer_path+"tokenizer_inputevents_mv.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['inputevents_mv'],handle) 

#saving the Labevents tokenizer to disk
with open(tokenizer_path+"tokenizer_labevents.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['labevents'],handle) 
    
#saving the Microbioevents tokenizer to disk
with open(tokenizer_path+"tokenizer_microbioevents.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['microbioevents'],handle) 
    
#saving the Notevents tokenizer to disk
with open(tokenizer_path+"tokenizer_noteevents.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['noteevents'],handle) 

#saving the Outputevents tokenizer to disk
with open(tokenizer_path+"tokenizer_outputevents.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['outputevents'],handle) 

#saving the Prescriptionevents tokenizer to disk
with open(tokenizer_path+"tokenizer_prescriptionevents.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['prescriptionevents'],handle) 
    
#saving the Procedureevents tokenizer to disk
with open(tokenizer_path+"tokenizer_procedureevents.pickle", 'wb') as handle:
    pickle.dump(tokenizer_dict['procedureevents'],handle) 

### Integer Encode Train Data, Valid Data, Test Data

In [43]:
# function to remove low count words from tokenizer
def tokenizer_low_count(tokenizer):
    t = tokenizer
    #initialize unknown token
    t.oov_token = 'UNK'
    # add "UNK" token and an integer value for it to the tokenizer word index
    t.word_index.update({'UNK':len(t.word_index)+1})
    # Create a list of tokens that occur only once
    low_count_words = []
    for k,v in t.word_counts.items():
        if v==1:
            low_count_words.append(k)
    #Removed words with low count from tokenizer
    for w in low_count_words:
        del t.word_index[w]
    return t

In [54]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_procedureevents.pickle", 'rb') as handle:
    t = pickle.load(handle)

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['procedureevents'] = df_train['procedureevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['procedureevents'] = df_valid['procedureevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['procedureevents'] = df_test['procedureevents'].map(lambda y: t.texts_to_sequences([y])[0])

In [55]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_chartevents.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['event'] = df_train['event'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['event'] = df_valid['event'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['event'] = df_test['event'].map(lambda y: t.texts_to_sequences([y])[0])

In [56]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_inputevents_cv.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['inputevents_cv'] = df_train['inputevents_cv'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['inputevents_cv'] = df_valid['inputevents_cv'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['inputevents_cv'] = df_test['inputevents_cv'].map(lambda y: t.texts_to_sequences([y])[0])

In [57]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_inputevents_mv.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['inputevents_mv'] = df_train['inputevents_mv'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['inputevents_mv'] = df_valid['inputevents_mv'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['inputevents_mv'] = df_test['inputevents_mv'].map(lambda y: t.texts_to_sequences([y])[0])

In [58]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_labevents.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['labevents'] = df_train['labevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['labevents'] = df_valid['labevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['labevents'] = df_test['labevents'].map(lambda y: t.texts_to_sequences([y])[0])

In [59]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_microbioevents.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['microbioevents'] = df_train['microbioevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['microbioevents'] = df_valid['microbioevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['microbioevents'] = df_test['microbioevents'].map(lambda y: t.texts_to_sequences([y])[0])

In [60]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_noteevents.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['noteevents'] = df_train['noteevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['noteevents'] = df_valid['noteevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['noteevents'] = df_test['noteevents'].map(lambda y: t.texts_to_sequences([y])[0])

In [61]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_outputevents.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['outputevents'] = df_train['outputevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['outputevents'] = df_valid['outputevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['outputevents'] = df_test['outputevents'].map(lambda y: t.texts_to_sequences([y])[0])

In [62]:
# loading the tokenizer form disk
with open(tokenizer_path+"tokenizer_prescriptionevents.pickle", 'rb') as handle:
    t = pickle.load(handle) 

# Removing tokens with low count
tokenizer_low_count(t)

# replacing tokens with their integer codes
df_train['prescriptionevents'] = df_train['prescriptionevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_valid['prescriptionevents'] = df_valid['prescriptionevents'].map(lambda y: t.texts_to_sequences([y])[0])
df_test['prescriptionevents'] = df_test['prescriptionevents'].map(lambda y: t.texts_to_sequences([y])[0])

In [63]:
#save the integer encoded data to disk
# create output path
file_save_path = '/home/jupyter/datasets/training_data/data_before_24hrs_icu/data_grouped_HADM_ID/train_test_valid/'
import os
os.makedirs(file_save_path, exist_ok=True)
# export the dataframe to JSON format
df_train.to_json(file_save_path+"all_events_train.json",orient = 'records')
df_valid.to_json(file_save_path+"all_events_valid.json",orient = 'records') 
df_test.to_json(file_save_path+"all_events_test.json",orient = 'records') 